In [ ]:
#prep
import os
!pip install tqdm -q
from tqdm.notebook import tqdm
from IPython.display import clear_output
from os.path import exists, join, basename, splitext
!pip install resampy
!pip install git+https://github.com/IAHispano/gdown.git
git_repo_url = 'https://github.com/justinjohn0306/TTS-TT2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
# clone and install
  !git clone -q --recursive {git_repo_url}
  !git clone -q --recursive https://github.com/justinjohn0306/hifi-gan
  !pip install -q unidecode
import sys
sys.path.append('hifi-gan')
sys.path.append(project_name)


import logging

logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('librosa').setLevel(logging.WARNING)


#Universal HiFi-GAN (has some robotic noise): 1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW

#@markdown ---


#@markdown **The "tacotron_id" is where you can put a link to your trained tacotron2 model from Google Drive.**


#@markdown If the audio sounds too artificial, you can lower the superres_strength

#@markdown ---
#@markdown Config:

#@markdown Restart the runtime to apply any changes.

tacotron_id = "1IElD24WJEiiY5GO4LMJTQJWtGjmo_qfb" #@param {type:"string"}

hifigan_id = "1-JWgMJ0RS__dBhdpjscxwe4xJ7_mA0HK" #@param {type:"string"}

#@markdown _leave blank or enter "universal" for universal model_

#@markdown ---

if tacotron_id != "":
    TACOTRON2_ID = tacotron_id
else:
    raise Exception("No ID provided.")

if hifigan_id in {"", "universal"}:
    HIFIGAN_ID = "universal"
    print("Using universal Hifi-Gan model.")
else:
    HIFIGAN_ID = hifigan_id

# Check if Initialized
try:
    initialized
except NameError:
    print("Setting up, please wait.\n")
    with tqdm(total=5, leave=False) as pbar:

        import time
        import matplotlib
        import matplotlib.pylab as plt
        import gdown
        d = 'https://drive.google.com/uc?id='

        %matplotlib inline
        import IPython.display as ipd
        import numpy as np
        import torch
        import json
        from hparams import create_hparams
        from model import Tacotron2
        from layers import TacotronSTFT
        from audio_processing import griffin_lim
        from text import text_to_sequence
        from env import AttrDict
        from meldataset import mel_spectrogram, MAX_WAV_VALUE
        from models import Generator
        from denoiser import Denoiser
        import resampy
        import scipy.signal

        pbar.update(1) # initialized Dependancies

        graph_width = 900
        graph_height = 360
        def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
            %matplotlib inline
            fig, axes = plt.subplots(1, len(data), figsize=figsize)
            for i in range(len(data)):
                axes[i].imshow(data[i], aspect='auto', origin='lower',
                            interpolation='none', cmap='inferno')
            fig.canvas.draw()
            plt.show()

        # Setup Pronounciation Dictionary
        !wget 'https://github.com/justinjohn0306/FakeYou-Tacotron2-Notebook/releases/download/CMU_dict/merged.dict.txt'
        thisdict = {}
        for line in reversed((open('merged.dict.txt', "r").read()).splitlines()):
            thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()

        pbar.update(1) # Downloaded and Set up Pronounciation Dictionary

        def ARPA(text, punctuation=r"!?,.;", EOS_Token=True):
            out = ''
            for word_ in text.split(" "):
                word=word_; end_chars = ''
                while any(elem in word for elem in punctuation) and len(word) > 1:
                    if word[-1] in punctuation: end_chars = word[-1] + end_chars; word = word[:-1]
                    else: break
                try:
                    word_arpa = thisdict[word.upper()]
                    word = "{" + str(word_arpa) + "}"
                except KeyError: pass
                out = (out + " " + word + end_chars).strip()
            if EOS_Token and out[-1] != ";": out += ";"
            return out

        def get_hifigan(MODEL_ID, conf_name):
            # Download HiFi-GAN
            hifigan_pretrained_model = 'hifimodel_' + conf_name
            #gdown.download(d+MODEL_ID, hifigan_pretrained_model, quiet=False)

            if MODEL_ID == 1:
              !wget "https://github.com/justinjohn0306/tacotron2/releases/download/assets/Superres_Twilight_33000" -O $hifigan_pretrained_model
            elif MODEL_ID == "universal":
              !wget "https://github.com/justinjohn0306/tacotron2/releases/download/assets/g_02500000" -O $hifigan_pretrained_model
            else:
              gdown.download(d+MODEL_ID, hifigan_pretrained_model, quiet=False)

            if not exists(hifigan_pretrained_model):
                raise Exception("HiFI-GAN model failed to download!")

            # Load HiFi-GAN
            conf = os.path.join("hifi-gan", conf_name + ".json")
            with open(conf) as f:
                json_config = json.loads(f.read())
            h = AttrDict(json_config)
            torch.manual_seed(h.seed)
            hifigan = Generator(h).to(torch.device("cpu"))
            state_dict_g = torch.load(hifigan_pretrained_model, map_location=torch.device("cpu"))
            hifigan.load_state_dict(state_dict_g["generator"])
            hifigan.eval()
            hifigan.remove_weight_norm()
            denoiser = Denoiser(hifigan, mode="normal")
            return hifigan, h, denoiser

        # Download character HiFi-GAN
        hifigan, h, denoiser = get_hifigan(HIFIGAN_ID, "config_v1")
        # Download super-resolution HiFi-GAN
        hifigan_sr, h2, denoiser_sr = get_hifigan(1, "config_32k")
        pbar.update(1) # Downloaded and Set up HiFi-GAN

        def has_MMI(STATE_DICT):
            return any(True for x in STATE_DICT.keys() if "mi." in x)

        def get_Tactron2(MODEL_ID):
            # Download Tacotron2
            tacotron2_pretrained_model = 'MLPTTS'
            gdown.download(d+MODEL_ID, tacotron2_pretrained_model, quiet=False)
            if not exists(tacotron2_pretrained_model):
                raise Exception("Tacotron2 model failed to download!")
            # Load Tacotron2 and Config
            hparams = create_hparams()
            hparams.sampling_rate = 22050
            hparams.max_decoder_steps = 3000 # Max Duration
            hparams.gate_threshold = 0.25 # Model must be 25% sure the clip is over before ending generation
            model = Tacotron2(hparams)
            state_dict = torch.load(tacotron2_pretrained_model, map_location=torch.device("cpu"))['state_dict']
            if has_MMI(state_dict):
                raise Exception("ERROR: This notebook does not currently support MMI models.")
            model.load_state_dict(state_dict)
            _ = model.eval()
            return model, hparams

        model, hparams = get_Tactron2(TACOTRON2_ID)
        previous_tt2_id = TACOTRON2_ID

        pbar.update(1) # Downloaded and Set up Tacotron2

        # Extra Info
        def end_to_end_infer(text, pronounciation_dictionary, show_graphs):
            for i in [x for x in text.split("\n") if len(x)]:
                if not pronounciation_dictionary:
                    if i[-1] != ";": i=i+";"
                else: i = ARPA(i)
                with torch.no_grad(): # save VRAM by not including gradients
                    sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
                    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).long()
                    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
                    if show_graphs:
                        plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                                alignments.float().data.cpu().numpy()[0].T))
                    y_g_hat = hifigan(mel_outputs_postnet.float())
                    audio = y_g_hat.squeeze()
                    audio = audio * MAX_WAV_VALUE
                    audio_denoised = denoiser(audio.view(1, -1), strength=35)[:, 0]

                    # Resample to 32k
                    audio_denoised = audio_denoised.cpu().numpy().reshape(-1)

                    normalize = (MAX_WAV_VALUE / np.max(np.abs(audio_denoised))) ** 0.9
                    audio_denoised = audio_denoised * normalize
                    wave = resampy.resample(
                        audio_denoised,
                        h.sampling_rate,
                        h2.sampling_rate,
                        filter="sinc_window",
                        window=scipy.signal.windows.hann,
                        num_zeros=8,
                    )
                    wave_out = wave.astype(np.int16)

                    # HiFi-GAN super-resolution
                    wave = wave / MAX_WAV_VALUE
                    wave = torch.FloatTensor(wave).to(torch.device("cpu"))
                    new_mel = mel_spectrogram(
                        wave.unsqueeze(0),
                        h2.n_fft,
                        h2.num_mels,
                        h2.sampling_rate,
                        h2.hop_size,
                        h2.win_size,
                        h2.fmin,
                        h2.fmax,
                    )
                    y_g_hat2 = hifigan_sr(new_mel)
                    audio2 = y_g_hat2.squeeze()
                    audio2 = audio2 * MAX_WAV_VALUE
                    audio2_denoised = denoiser(audio2.view(1, -1), strength=35)[:, 0]

                    # High-pass filter, mixing and denormalizing
                    audio2_denoised = audio2_denoised.cpu().numpy().reshape(-1)
                    b = scipy.signal.firwin(
                        101, cutoff=10500, fs=h2.sampling_rate, pass_zero=False
                    )
                    y = scipy.signal.lfilter(b, [1.0], audio2_denoised)
                    y *= superres_strength
                    y_out = y.astype(np.int16)
                    y_padded = np.zeros(wave_out.shape)
                    y_padded[: y_out.shape[0]] = y_out
                    sr_mix = wave_out + y_padded
                    sr_mix = sr_mix / normalize

                    print("")
                    ipd.display(ipd.Audio(sr_mix.astype(np.int16), rate=h2.sampling_rate))
    from IPython.display import clear_output
    clear_output()
    initialized = "Ready"

if previous_tt2_id != TACOTRON2_ID:
    print("Updating Models")
    model, hparams = get_Tactron2(TACOTRON2_ID)
    hifigan, h, denoiser = get_hifigan(HIFIGAN_ID, "config_v1")
    previous_tt2_id = TACOTRON2_ID

pronounciation_dictionary = False #@param {type:"boolean"}
# disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing
show_graphs = False #@param {type:"boolean"}
max_duration =  10#@param {type:"integer"}
model.decoder.max_decoder_steps = max_duration * 80
stop_threshold = 0.3 #@param {type:"number"}
model.decoder.gate_threshold = stop_threshold
superres_strength = 6 #@param {type:"number"}

print(f"Current Config:\npronounciation_dictionary: {pronounciation_dictionary}\nshow_graphs: {show_graphs}\nmax_duration (in seconds): {max_duration}\nstop_threshold: {stop_threshold}\nsuperres_strength: {superres_strength}\n\n")

time.sleep(1)
contents = []
# Define the data (use your complete input as needed)
data = """wavs/clip_slide3_0000-0010.wav|Let's see some examples. So first, right is above the domain adaptation. So in this slide, we will see our
wavs/clip_slide3_0070-0080.wav|We have another kind of datasets in the medical field. For example, we have the t one images and we have the flare images if we
wavs/clip_slide4_0000-0010.wav|And I will introduce one more of the previous that I developed out of those examples. That's exactly how manifold
wavs/clip_slide5_0030-0040.wav|In this case, it's just a 16. And there's a out is the outer is just about, you know, 1, the shape of your brain structure.
wavs/clip_slide7_0020-0030.wav|Different causes of the diseases. If it is one, it means normal. If it is two, it means it's abnormal. So sometimes you
wavs/clip_slide9_0010-0020.wav|Left hand, you see a lot of, like, noise. You just, you know, bring it over. But on your right hand, once you get the denoised,
wavs/clip_slide10_0010-0020.wav|Here. Right? And the same on the left hand, the this figure is about noisy mixing birds. By the right hand, there are 2 different colors. So
wavs/clip_slide11_0040-0042.wav|And there's this one is a very pretty big.
wavs/clip_slide13_0030-0039.wav|And then this here is just our Canvas page, and then you can feel free to have some discussions online.
wavs/clip_slide15_0030-0034.wav|In the office after any kind of questions. Okay?
wavs/clip_slide18_0040-0050.wav|It's about thirty percentage for that reason, your final project is like your first semester first year come to my office, talk to me to
wavs/clip_slide18_0130-0140.wav|Yes. We, depending on your guys. If you guys sound like, other at the most, I would like to say two students per group.
wavs/clip_slide20_0010-0020.wav|And data pre processing in the 3rd way, we are talking about the classification after that, we will talk about some regression and list
wavs/clip_slide20_0060-0070.wav|We have, like, neural network, Anselm method, and we are finished about the horse method, I'll have to emphasize that soon.
wavs/clip_slide20_0130-0140.wav|The final project, professor. Yes. Let's go try to find it in our syllabus again. So here's all the content that I will cover.
wavs/clip_slide20_0160-0170.wav|Are all the projects independent for that or, like, a teamwork professor? You mean the homework student wanna know the project project?
wavs/clip_slide20_0220-0230.wav|In your interview, how you will implement the scale? These kinds of algorithms like Gaussian Mixture Model, how you will implement?
wavs/clip_slide20_0250-0260.wav|Immediately implemented, and you definitely have a good impression of them. So now the, might need some might hire you immediately, hopefully. Okay.
wavs/clip_slide22_0040-0050.wav|What about the other source? This is so yeah. It's still kinda often. What about the other source of of from online students?
wavs/clip_slide24_0000-0010.wav|So first of all, was the perception of we also say it as, representation so in AI, perception
wavs/clip_slide25_0000-0010.wav|But what about learning? So the learning is the ability of a system to improve its behavior based on experience. So
wavs/clip_slide27_0020-0030.wav|Level and applying it at other level, so then another question, so among those four different aspects, Which
wavs/clip_slide27_0080-0082.wav|Answer.
wavs/clip_slide29_0030-0040.wav|Verification and validation is more like you like your phone.
wavs/clip_slide29_0100-0110.wav|Especially the amount that data labeling is a big challenge for all existing machine learning. For those, you definitely need to provide feed
wavs/clip_slide29_0160-0170.wav|Interaction, the HCI and Israeli would like a little bit longer time than single computer models, but you know
wavs/clip_slide30_0060-0070.wav|Label the input data. So then one features and labels, professor, feature and labels cn, that's yes. Exactly.
wavs/clip_slide30_0130-0140.wav|You've got some stock price predictions, so the stock price on your level our levels of it, right, giving you
wavs/clip_slide32_0020-0030.wav|Of that in the nineteen sixties, like the earliest offer network, it's called perception as it was born at that time.
wavs/clip_slide33_0020-0028.wav|Finding some tasks and learning, some reinforcement learning and inductive logic programming, etcetera.
wavs/clip_slide34_0030-0040.wav|About two years ago in the two thousand tens. In that, stage, and like, deep learning is rarely and becomes popular. Right?
wavs/clip_slide37_0000-0010.wav|So this is about the intuition of showing how it handles themselves. Actually, there are no specific features that
wavs/clip_slide37_0060-0070.wav|Yes. Right? What about the world? And that's about our children, so really humanness learn from experience.
wavs/clip_slide38_0030-0040.wav|Model is bigger, professor. Yes. That's very good spectrum. What about some thoughts about you guys use tradition?
wavs/clip_slide38_0100-0110.wav|Sum of if a statement and a check if that's related to a cat or not, professor. Good. You have a similar
wavs/clip_slide39_0030-0040.wav|We want to detect the shape, we want to detect the length, and we have so many different if else statements. So finally to decide
wavs/clip_slide41_0000-0010.wav|Very challenging. I mean, different shape. Strange shape somehow has a cat more like water. It can be in any shape. Right?
wavs/clip_slide43_0000-0010.wav|So now we talk about the questions some what is machine learning? And this is very important The early stage of computer
wavs/clip_slide44_0020-0030.wav|Because we are human beings. Right? And we are we were lazy. We don't we cannot do everything. We want some computers
wavs/clip_slide46_0000-0010.wav|The next question is, why study machine learning? Why is it important? Here's the case in that we want to develop a better computer system.
wavs/clip_slide46_0060-0070.wav|Users. For example, personalized news, the mailing filtering, for example, sometimes you have some app.
wavs/clip_slide47_0010-0020.wav|Learn is not intelligent. So, actually, without learning, everything is new. Right? A system that cannot learn is not efficient because it
wavs/clip_slide48_0000-0010.wav|You will try to use machine learning For example, in the speech recognition for example, you will use Siri and ask her to do
wavs/clip_slide51_0000-0010.wav|And, also, we can recognize, handwritten digits. For example, this kind of very weird. Right? It's it is weird. Right?
wavs/clip_slide54_0010-0017.wav|Been there before, right? Who can use some robot machine help us be explored first, right?
wavs/clip_slide56_0000-0010.wav|Another trend is about why machine learning models are so popular. Right? Because, there are so many places that we needed to use machine learning
wavs/clip_slide56_0090-0097.wav|To use machine learning, and this is the key reason why machine learning nowadays is so popular and important.
wavs/clip_slide58_0020-0030.wav|Learning we have we will use labeled dataset, right, as the unsupervised learning just to discover patterns in unlabeled dataset.
wavs/clip_slide59_0010-0020.wav|Had a semi supervised. Another count between those ones, As a majority, we have three those three different categories, for example, is
wavs/clip_slide59_0050-0060.wav|Unlabeled data is a traditional sense is a reinforcement learning. An agent learns via its interactions with any environment.
wavs/clip_slide60_0010-0020.wav|The perception, especially about logistic regression. I think all of you should know about from your first semester. Right?
wavs/clip_slide60_0070-0080.wav|The non parametric models they just they need just that those models vary depending on dataset they don't care
wavs/clip_slide60_0120-0128.wav|For majority of those kinds of methods, we will cover the rest of the semester, you will see about that.
wavs/clip_slide62_0050-0060.wav|Predictor labels. Right? However, why important the process is called feature extraction? Well done.
wavs/clip_slide62_0100-0110.wav|Method to extract some key features from data So here, it is the images. The key feature
wavs/clip_slide62_0160-0163.wav|You will get more sense of it.
wavs/clip_slide63_0040-0050.wav|Based on some clustering method. Right? For example, that like some knn k nearest neighbor, you will probably choose.
wavs/clip_slide63_0090-0097.wav|Those two different classes, this is, cool called unsupervised learning.
wavs/clip_slide64_0090-0100.wav|Anyway, you guys will learn more details that in the AI, artificial intelligence class, or reinforcement learning class, and specifically for
wavs/clip_slide65_0010-0020.wav|We only utilize a kind of traditional machine learning models. For example, I like the decision tree, the SVM, the KAN, The MLP
wavs/clip_slide66_0010-0020.wav|Works support vector machine and the symbolic functions, we have decision trees, processional logic, rules for the first
wavs/clip_slide68_0000-0010.wav|For example, this is very about the medical diseases diagnosis is left for about this image. Any idea about this image?
wavs/clip_slide69_0010-0020.wav|Your eyes, where is your nose? Where is your mouse? Right? It's funny areas. Right? It's really about personal identification. So
wavs/clip_slide71_0010-0012.wav|just tries to shoot a posture.
wavs/clip_slide76_0030-0040.wav|Right? And there's that's there's some repetitive tasks for some tasks that you will repeatedly do it so
wavs/clip_slide77_0020-0028.wav|Example, like packaging, the first need is to recollect objects. So then you can package those objects.
wavs/clip_slide82_0010-0018.wav|And also about military suits, or about this case try to do some detections, some problem of it.
wavs/clip_slide84_0040-0050.wav|You do not want to break it, either to break, the object or to break the robot itself. What about this cleaner?
wavs/clip_slide87_0000-0010.wav|And this is a famous sentence from Birgitte that a breakthrough in machine learning will be worth 10 Microsofts. Is that
wavs/clip_slide89_0040-0050.wav|For example, you guys have such a lot of those questions, like machine learning, and then that's the key reason you guys need to start in this.
wavs/clip_slide90_0040-0050.wav|Remind this class, I will require you guys also to provide some answers from chat gbt answers. Okay? So, basically, which means that
wavs/clip_slide91_0000-0010.wav|So for example, here is the example from me from me. And since this is for the examples from the chat gpt, but the question, what is
wavs/clip_slide91_0070-0080.wav|Well, you cannot beat a machine. Right? You cannot beat the a chat gbt with a good answer. Right? Try to avoid that. Okay. Others probably
wavs/slide31.wav|So next, I want to briefly mention about the history of the Machine Learning.
wavs/slide83.wav|And also, you know about the space robot that I mentioned before to try to explore the Mars. Right?"""

# Split the input data by lines
lines = data.strip().split('\n')

# Extract the text part (after the '|')
sentences = [line.split('|')[1] for line in lines]

for i in sentences:
    try:
        print("-"*50)
        line = i
        if line == "":
            continue
        end_to_end_infer(line, not pronounciation_dictionary, show_graphs)
    except EOFError:
        break
    except KeyboardInterrupt:
        print("Stopping...")
        break


In [3]:
import torch
import torchaudio
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility
import pysepm
from pesq import pesq
from pystoi import stoi
from scipy.io import wavfile
import sys
sys.path.append("../")

# Initialize PESQ and STOI with 16kHz sample rate
pesq = PerceptualEvaluationSpeechQuality(16000, 'wb')
stoi = ShortTimeObjectiveIntelligibility(16000)


# Data to process
data = """wavs/1.wav|AKAN GRAVES TODAY HER GRANDDAUGHTER STEPHANI SERVES AS A GUIDE AT THIS CEMETERY THIS WEEK
wavs/2.wav|E DAVID ARRAYED ON THESE BEAUTIFUL GROUNDS EACH ONE HAS BEEN ADOPTED BY A FRENCH FAMILY THAT THINKS
wavs/3.wav|PLAYS FLOWERS AND THEY NEVER FORGET TO DAY AMERICA EMBRACES THE FRENCH PEOPLE AND THANKS YOU FOR
wavs/4.wav|IS  UNBREAKABLE FROM ACROSS THE EARTH AMERICANS ARE DRAWN TO THIS PLACE AS THOUGH IT WERE A PART OF OUR VERY SOUL WE
wavs/5.wav|CUM NOT ONLY BECAUSE OF WHATT THEY DID HEAR WE COME BECAUSE OF WHO THEY WERE THEY WERE YOUNG MEN
wavs/6.wav|WITH THEIR ENTIRE LIVES BEFORE THEM THEY WERE HUSBANDS WHO SAID GOOD BY TO THEIR YOUNG BRIDES AND TOOK THEIR DUTY AS
wavs/7.wav|THEIR FATE THEY WERE FATHERS WHO WOULD NEVER MEET THEIR INFINITE SONS AND DAUGHTERS BECAUSE THEY HAD A JOB TO DO AND WITH GOD AS THEIR WITNESS
wavs/8.wav|THOUT A COMPLAINT MORE POWERFUL THAN THE STRENGTH OF AMERICAN ARMS WAS THE STRENGTH OF AMERICAN HORACE
wavs/9.wav|THEY PRESSED ON FOR LOVE AND HOME AND COUNTRY THE MAIN STREETS THE SCHOOLYARDS THE CHURCHES AND NEIGHBOURS THE FAMILIES AND COMMUNITIE
wavs/10.wav|THEY GAVE US MEN SUCH AS THESE THEY WERE SUSTAINED BY THE CONFIDENCE THAT AMERICA CAN DO ANYTHING BECAUSE WE ARE A NOBLE NATIO
wavs/11.wav|OF GOD'S HAND THE MEN BEHIND ME WILL TELL YOU THAT THEY ARE JUST THE LUCKY ONES AS
wavs/12.wav|S ONE OF THEM RECENTLY BEHOODED ALL THE HEROES ARE BURIED HERE BUT WE KNOW WHAT THESE MEN DID
wavs/13.wav|WHAT FREEDOMM IS ALL ABOUT THE AMERICAN SONS AND DAUGHTERS WHO SAW US TO VICTORY WERE NO LESS EXTRAORDINAR
wavs/14.wav|REVOLUTIONIZED SCIENCE LAUNCHED A MAN TO THE MOON AND THEN KEPT ON PUSHING TO NEW FRONTIERS
wavs/15.wav|HERE WITH YOU ARE OVER SIXTY VETERANS WHO LANDED ON D DAY OUR DEBT TO YOU IS EVERLASTING TO DAY WE EXPRESS OUR UNDYING
wavs/16.wav|HO SPOKE OF A THOUSAND YEAR EMPIRE IN DEFEATING THAT EVIL THEY LEFT A LEGACY THAT WILL LAST NOT ONLY FOR A THOUSAND YEARS
wavs/17.wav|THE BLOOD THAT THEY SPILLED THE TEARS THAT THEY SHED THE LIVES THAT THEY GAVE THE SACRIFICE THAT THE
wavs/18.wav|OR NATION THEY WON THE SURVIVAL OF OUR CIVILIZATION AND THEY SHOWED US THE WAY TO LOVE CHERI
wavs/19.wav|GRATITUDE WHEN YOU WERE YOUNG THESE MEN ENLISTED THEIR LIVES IN A GREAT CRUSADE ONE OF THE GREATEST OF ALL TIMES THEI
wavs/20.wav|AND DEFEND OUR WAY OF LIFE FOR MANY CENTURIES TO COME TO DAY AS WE STAND TOGETHER UPON THIS SAINT CRITIC
wavs/21.wav|EARTH WE PLEDGE THAT OUR NATION WILL FOREVER BE STRONG AND UNITED WE WILL FOREVER B
wavs/22.wav|MISSION IS THE STORY OF AN EPOC BATTLE AND THE FEROCIOUS ETERNAL STRUGGLE BETWEEN GOOD AND EVIL ON THE SIXTH OF JUNE NINETEEN FORTY FOUR
wavs/23.wav|THEY JOINED A LIBERATION FORCE OF AWESOME POWER AND BREATH TAKING SCALE AFTER MONTHS OF PLANTING THE ALLIES HAD CHOSEN THIS ANCIENT CO
wavs/24.wav|ST LINE TO MOUNT THEIR CAMPAIGN TO VANQUISH THE WICKED TYRANNY OF THE NAZSI EMPIRE FROM THE FACE OF THE EARTH THE BATTLE B
wavs/25.wav|AGAIN IN THE SKIES ABOVE US IN THOSE FIRST TENS MIDNIGHT HOURS ONE THOUSAND AIRCRAFT ROARED OVERHEAD WITH SEVENTEEN THOUSAND ALL
wavs/26.wav|EYED AERBORN TROOPS PREPARING TO LEAP INTO THE DARKNESS BEYOND THESE TREES THEN CAME DAWN THE ENEMY WHO HAD OCCUPIED THESE HEIGHTS
wavs/27.wav|SAW THE LARGEST NAVAL ARMADA IN THE HISTORY OF THE WORLD JUST A FEW MILES OFF SHORE WERE SEVEN THOUSAND VESSELS BEARING ONE
wavs/28.wav|SSUS MACCROWN AND THE PEOPLE OF FRANCE TO THE FIRST LADY OF THE UNITED STATES AND MEMBERS OF THE UNI
wavs/29.wav|HUNDRED AND THIRTY THOUSAND WARRIORS THEY WERE THE CITIZENS OF FREE AND INDEPENDENT NATIONS UNITED BY THEIR DUTY TO THEIR COMPATRIOTS
wavs/30.wav|INTO MILLIONS YET UNBORN THERE WERE THE BRITISH WHOSE NOBILITY AND FORTITUDE SAW THEM THROUGH THE WORST OF
wavs/31.wav|UNKIRK AND THE LONDON BLITS THE FULL VIOLENCE OF NAUNSEY FURY WAS NO MATCH FOR THE FULL GRANDEUR OF
wavs/32.wav|THERE WERE THE CANADIANS WHOSE ROBUST SENSE OF HONOUR AND LOYALTY COMPEL THEM TO TAKE UP ARMS ALONGSIDE BRITAIN COM THE VERY VERY BEGINNING
wavs/33.wav|THERE WERE THE FIGHTING POLES THE TOUGH NORWEGIANS AND THE INTREPID AUSES THERE WERE THE GALLANT FRENCH
wavs/34.wav|MANDAS SOON TO BE MET BY THOUSANDS OF THEIR BRAVE COUNTRYMEN READY TO WRITE A NEW CHAPTER IN THE LONG HISTORY OF FRENCH VALOR
wavs/35.wav|AMERICANS THEY CAME FROM THE FARMS OF A VAST HEARTLAND THE STREETS OF GLOWING CITIES AND THE FORGES OF MIGHTY
wavs/36.wav|INDUSTRIAL TOWNS BEFORE THE WAR MANY HAD NEVER VENTURED BEYOND THEIR OWN COMMUNITY NOW THEY HAD COME TO OFFER THEIR LIVES
wavs/37.wav|TED STATES CONGRESS TO DISTINGUISH GASSS VETERANS AND MY FELLOW AMERICANS WE ARE GATHERED HERE
wavs/38.wav|HALF A WORLD FROM HOME THIS BEACH CODE NAMED OMAHA WAS DEFENDED BY THE N
wavs/39.wav|ATSIUS WITH MONSTROUS FIRE POWER THOUSANDS AND THOUSANDS OF MINES AND SPIKES DRIVEN INTO THE SAND SO DEEPLY
wavs/40.wav|IT WAS HERE THAT TENS OF THOUSANDS OF THE AMERICANS CAME THE GIIS WHO BOARDED THE LANDING CRAFT THAT MORNING KNEW THAT THEY C
wavs/41.wav|CARRIED ON THEIR SHOULDERS NOT JUST THE PACK OF A SOLDIER BUT THE FATE OF THE WORLD COLONEL GEORGE TAYLOR WHO
wavs/42.wav|SIXTEENTH INFANTRY REGIMENT WOULD JOIN IN THE FIRST WAIF WAS ASKED WHAT WOULD HAPPEN IF THE GERMAN STOPPED RIGHT
wavs/43.wav|HEN AND THERE COLD ON THE BEACH JUST STOP TEM WHAT WOULD HAPPEN THIS GREAT AMERICAN REPLIED WHY THE EIGHTEENTH INFANNTRY IS COMING IN RIGHT BE
wavs/44.wav|HIND US THE TWENTY SIXTH INFANTRY WILL COME ON TO THEN THERE IS THE SECOND INFANTRY DIVISION ALREADY AFLOAT A
wavs/45.wav|THE NINTH DIVISION AND THE SECOND ARMOURED AND THE THIRD ARMOURED AND ALL THE REST MAYBE THE SIXTEENTH WON'T MAKE IT BUT SOMEONE WILL ONE OF
wavs/46.wav|THOSE MEN IN TAILOR'S SIXTEENTH REGIMENT WAS ARMY METTICK RAY LAMBERT RAY WAS ONLY TWENTY THREE BUT HE HAD ALREADY EARN
wavs/47.wav|THREE PURPLE HEARTS AND TWO SILVER STARS FIGHTING IN NORTH AFRICA AND SICILY WHERE HE AND HIS BROTHER BILL NO LONGER WITH US SERVED SIDE BY
wavs/48.wav|ON FRIEDAM'S ALTAR ON THESE SHORES ON THESE BLUFFS ON THIS DAY SEVENTY FIVE YEARS AGO TEN THOUSAND MEN SHED THEIR
wavs/49.wav|SIDE IN THE EARLY MORNING HOURS THE TWO BROTHERS STOOD TOGETHER ON THE DECK OF THE U S S HINRICE
wavs/50.wav|EFOREBOARDING TWO SEPARATE HIGGINS LANDING CRAFT IF I DON'T MAKE IT BILL SAID PLEASE PLEASE TAKE CARE OF MY
wavs/51.wav|FAMILY RAY ASKED HIS BROTHER TO DO THE SAME OF THE THIRTY ONE MEN ONE RAISED LANDING CRAFT ONLY RAY AND SIX
wavs/52.wav|LAZERS MADED TO THE BEACH THERE WERE ONLY A FEW OF THEM LEFT THEY CAME TO THE SECTRE RIGHT HERE BELOW US
wavs/53.wav|EASY RED IT WAS CALLED AGAIN AND AGAIN RAY RAN BACK INTO THE WATER HE DRAGGED OUT ONE MAN AFTER ANOTHER
wavs/54.wav|HE WAS SHOT THROUGH THE ARM HIS LEG WAS RIPPED OPEN BY SHRAPNEL HIS BACK WAS BROKEN
wavs/55.wav|HE NEARLY DROWNED HE HAD BEEN ON THE BEACH FOR HOURS BLEEDING AND SAVING LIVES WHEN HE FINALLY LOST CONSCIOUSNESS
wavs/56.wav|HE WOKE UP THE NEXT DAY ON A COT BESIDE ANOTHER BADLY WOUNDED SOLDIER HE LOOKED OVER AND SAW HIS BROTHER BILL HE
wavs/57.wav|BLOOD AND THOUSANDS SACRIFICED THEIR LIVES FOR THEIR BROTHERS FOR THEIR COUNTRIES AND FOR THE SURVIVAL OF LIBERTY
wavs/58.wav|RELENTING FIRE FROM THESE BLUFFS KEPT THE AMERICANS PINNED DOWN ON THE SAND NOW RED WITH OUR HERO'S BLOOD THEN JUST A FEW HUNDRED YARDS FOM WERHAM'S
wavs/59.wav|DENDING A BREAK THROUGH CAME THE BATTLE TURNED AND WITH IT HISTORY DOWN ON THE BEACH CAPTAIN
wavs/60.wav|JOE DARSON THE SON OF A TEXAS PREACHER LED COMPANY G THROUGH A MINE FIELD TO A NATURAL FALD IN THE HILL
wavs/61.wav|SIDE STILL HERE JUST BEYOND THIS PATH TO MY RIGHT CAPTAIN DARSON SNUCK BENEATH AN ENEMY MACHINE GUN PERCH AND TOSSED HIS GRENADES
wavs/62.wav|TO DAY WE REMEMBER THOSE WHO FELL AND WE HONOUR ALL WHO FOUGHT RIGHT HERE IN NORMANDY THEY WON BACK
wavs/63.wav|SOON AMERICAN TROOPS WERE CHARGING UP DARSAN'S DRAW WHAT A JOB HE DID WHAT BRAVERY HE SHOWED
wavs/64.wav|LIEUTENANT SPALDING AND THE MEN FROM COMPANY E MOVED ON TO CRUSH THE ENEMY'S STRONG POINT ON THE FAR SIDE OF THIS CEMETERY AND STOP THE S
wavs/65.wav|ATER ON THE BEACH BELOW COUNTLESS MOOR AMERICANS POURED OUT ACROSS THIS GROUND ALL OVER THE COUNTRYSIDE THEY JOINED FELL
wavs/66.wav|TO RETURN HE SAID I'M GOING TO RETURN SIX DAYS AFTER D DAY HE REJOINED HIS COMPANY TWO THIRDS HAD BEEN KI
wavs/67.wav|DFORD VIRGINIA ALONE BEFORE LONG A GRENADE LEFT PRIVATE PICKET AND HE WAS GRAVELY WOUNDED
wavs/68.wav|SO BADLY WON'T IT AGAIN HE CHOSE TO RETURN HE DIDN'T CARE HE HAD TO BE HERE HE WAS THEN WOUNDED A THIRD TIME AND LAID ON
wavs/69.wav|THIS GROUND FOR CIVILIZATION TO MORE THAN ONE HUNDRED AND SEVENTY VETERANS OF THE SECOND WORLD WAR WHO JOIN US TO DAY YOU ARE AMONG
wavs/70.wav|CONSCIOUS FOR TWELVE DAYS THEY THOUGHT HE WAS GONE THEY THOUGHT HE HAD NO CHANCE RUSSELL PICKET IS THE LAST KNOWN SURVIVOR OF THE LEGENDER
wavs/71.wav|TO LIBERATE JEWS WHO HAD SUFFERED THE BOTTOMLESS HORRORS OF THE HOLLACOST AND SOME WARRIORS FELL ON OTHER FIELDS OF BATTLE RETURNING
wavs/72.wav|THE VERY GREATEST AMERICANS WHO WILL EVER LIVE YOU ARE THE PRIDE OF OUR NATION YOU ARE THE GLORY OF OUR REPUBLIC
wavs/73.wav|TO REST ON THIS SOIL FOR ETERNITY BEFORE THIS PLACE WAS CONSECRATED TO HISTORY THE LAND WAS OWNED BY A FRENCH FARMER
wavs/74.wav|A MEMBER OF THE FRENCH RESISTANCE THESE WERE THE GREAT PEOPLE THESE WERE STRONG AND TOUGH PEOPLE HIS TERRIFIED WIFE
wavs/75.wav|MERICAN HE SAID I'M HERE TO HELL THE FRENCHWOMAN WAS OVERCOME WITH EMOTION AND CRIED DAYS LATER SHE LAID FLOWERS ON FRESH AMER
wavs/76.wav|THANK YOU TLAS VERY MUCH IT'S A PRIVILEGE TO BE HERE AT THE SFORM WHERE LEADERS IN BUSINESS SCIENCE ART DIPLOMACY AND WORLD AFFAIRS HAVE GATHERED FOR MANY MANY YEARS TO
wavs/77.wav|PEOPLE WE HAD DINNER LAST NIGHT WITH ABOUT FIFTEEN LEADERS OF INDUSTRY NONE OF WHOM I KNEW BUT ALL OF WHOM I'VE READ ABOUT FOR YEARS AND IT WAS TRULY AN INCREDIBL
wavs/78.wav|GROUP AND I THINK I HAVE FIFTEEN NEW FRIENDS SO THIS HAS BEEN REALLY GREAT WHAT YOU'VE DONE IN PUTTING IT TOGETHER THE ECONOMIC FORUM THE TAX REFORM WAS A DREAM
wavs/79.wav|OF LOT OF A LOT OF PEOPLE OVER MANY YEARS BUT THEY WERE UNTABLE TO GET IT DONE MANY PEOPLE TRIED AND RONALD RAGERD WAS REALLY THE LAST TO MAKE A MEANINGFUL CUT AND REFORM AND OURS IS CUTTING AND REFORM
wavs/80.wav|MING WE EMPHASIZE CUTT BUT THE REFORM IS PROBABLY ALMOST AS IMPORTANT WE'VE WANTED TO DO IT IT IS VERY TOUGH POLITICALLY TO DO IT HARD TO BELIEVE THAT WOULD BE
wavs/81.wav|T IT IS VERY VERY TOUGH THAT'S WHY IT HASN'T BEEN DONE IN CLOSE TO FORTY YEARS AND ONCE WE GOT IT GOING IT WAS GOING AND THE BIG AND I WOULDN'T SAY A TOTAL SURPRISE BUT ONE OF TH
wavs/82.wav|BIG THINGS THAT HAPPENED AND TOOK PLACE IS EIGHTY AND T AND SOME OTHERS CAME OUT VERY EARLY AND THEY SAID THEY WERE GOING TO PAY THOUSANDS AND THOUSANDS OF DOLLARS TO
wavs/83.wav|NEW ON EMPLOYMENT CLAIMS ARE NEAR THE LOWEST WE'VE SEEN IN ALMOST HALF A CENTURY AFRICAN AMERICAN UNEMPLOYMENT HAS REACHED THE LOWEST RATE EVER RECORDED
wavs/84.wav|PEOPLE THAT WORK FOR THEIR COMPANIES AND YOU HAVE THREE HUNDRED THOUSAND FOUR HUNDRED THOUSAND FIVE HUNDRED THOUSAND PEOPLE WORKING FOR THESE COMPANIES AND ALL OF A SUDDEN IT BECAME LIKE A BIG WATERFALL A BIG BEAUTIFUL WATERFALL
wavs/85.wav|WHERE SO MANY COMPANIES ARE DOING IT AND EVEN TO DAY THEY JUST ANNOUNCED MANY MORE BUT EVERY DAY THEY ANNOUNCE MORE AND MORE AND NOW IT'S A FIGHT FOR WHO'S GOING TO GIVE THE MOST ITS STARTED AT A THOUSAND NOW WE HAVE EM UP TO THREETH
wavs/86.wav|HOUSAND THIS IS SOMETHING THAT WE DIDN'T ANTICIPATE OFTENTIMES IN BUSINESS THINGS HAPPEN THAT YOU DON'T ANTICIPATE USUALLY THAT'S A BAD THING BUT THIS WAS A GOOD THING THIS
wavs/87.wav|CAME OUT OF NOWHERE NOBODY EVER THOUGHT OF THIS AS A POSSIBILITY EVEN WE IT WASN'T IN THE EQUATION WE WAITED WE SAID WHADLE FEBRUARY FIRST WHEN THE CHECKS START COMING IN AND PE
wavs/88.wav|LCLAUS HAVE A LOT MORE MONEY ON THEIR PAY CHACK CAUSE IT'S NOT JUST A LITTLE MONEY THERE'S A LOT OF MONEY FOR PEOPLE MAKING A LIVING DOING WHATEVER THEY MAY BE DOING AND WE REALLY THOUGHT FEBRUARY FIRST IT WAS GOING TO K
wavs/89.wav|CK IN AND EVERYBODY WAS GOING TO BE WOE WE HAVEN'T EVEN GOTTEN THERE YET AND IT'S KICKED IN AND IT'S HAD A INCREDIBLE IMPACT ON THE STOCK MARKET AND THE STOCK PRICES WE'VE SAID EIGHTY FOUR RECORDS
wavs/90.wav|SINCE MY ELECTION RECORD STOCK MARKET PRICES MEANING WE HIT NEW HIVES EIGHTY FOUR DIFFERENT TIMES OUT OF A ONE YEAR PERIOD AND THAT'S A GREA
wavs/91.wav|THING AND INOR FAIRNESS THAT WAS DONE BEFORE WE PASSED THE TAXI CUTS AND TAX REFORM SO WHAT HAPPENED IS REALLY SOMETHING SPECIAL THEN AS YOU KNOW AND AS I JUST SAID
wavs/92.wav|APPLE CAME IN WITH THREE HUNDRED AND FIFTY BILLION DOLLARS AND I TELL YOU AI SPOKE WITH TIM COOK I SAID TIM I WILL NEVER CONSIDER THIS WHOLE GREAT RUN THAT WE'VE MADE
wavs/93.wav|COMPLETE UNTIL YOU START BUILDING PLANTS IN THE U S AND I WILL TELL YOU THIS MOVED UP VERY SUBSTANTIALLY BUT WHEN I HEARD THREE HUNDRED AND FIFTY I THOUGHT HE WAS TALKING I THOUGHT THEY WERE TALKI
wavs/94.wav|DISCUSS HOW WE COULD ADVANCE PROSPERITY SECURITY AND PEACE I AM HERE TO DAY TO REPRESENT THE INTERESTS OF THE AMERICAN PEOPLE AND TO AFFIRM AMERICA'S FRIENDSHIP AND PARTNERSHI
wavs/95.wav|IN THE UNITED STATES AND SO HAS UNEMPLOYMENT AMONG HIS SPANIC AMERICANS THE WORLD IS WITNESSING THE RESURGENCE OF A STRONG
wavs/96.wav|THREE HUNDRED FIFTY MILLION DOLLARS AND BY THE WAY THAT'S A NICE SIZED PLANT NOT THE GREATEST BUT NOT BAD AND THEY SAID NO SIR IT'S THREE HUNDRED FIFTY MILLION DOLLAR
wavs/97.wav|I SAID THAT IS SOMETHING WELL WE HAVE TREMENDOUS AMOUNTS OF MONEY AND POLLUDING MY NEW FOUND FRIENDS FROM LAST NIGHT A GREAT COMPANIES THEY ARE ALL INVESTING ER WHEN ONE OF THE GE
wavs/98.wav|LEMEN SAID HE'S PUTTING IN TWO BILLION DOLLARS BECAUSE OF THE TAX CUTS I SAID TO MYSELF WOW HE'S ACTUALLY THE CHEAP ONE IN THE GROUP BECAUSE THEY'RE PUTTING IN MASSIVE NUMBERS OF BILLIONS OF DOLLARS SO
wavs/99.wav|I THINK YOU HAVE A BRAND NEW UNITED STATES YOU HAVE A UNITED STATES WHERE PEOPLE FROM ALL OVER THE WORLD ARE LOOKING TO COME AND AND INVEST AND THERE'S JUST NOTHING
wavs/100.wav|LIKE WHAT'S HAPPENING AND I JUST WANT TO FINISH BY I HAVE A A GROUP OF PEOPLE THAT HAVE BEEN SO OUT SO HAVE A WHOLE LOT OF EM SO I WON'T INTRODUCE CUS AND I'LL INSULT AT LEAST HALF OF EM BUT I'VE HAD A GROUP OF PEOPLE
wavs/101.wav|WORK SO HARD ON THIS AND OTHER THINGS AND WE'RE REALLY DOING WE HAT A GREAT FIRST YEAR SO SUCCESSFUL IN SO MANY DIFFERENT WAYS AND THERE'S A TREMENDOUS SPIRIT WHEN YOU LOOK AT
wavs/102.wav|ALL OF THE DIFFERENT CHARTS AND POLES AND YOU SEE A AS AN EXAMPLE AFRICAN AMERICAN ON EMPLOYMENT AT THE HISTORIC LOW THAT'S NEVER T'S NEVER HAD A PERIOD OF TIME LIKE THIS
wavs/103.wav|A SAME WITH HISPANIC WOMEN AT A SEVENTEEN YEAR LOW IT'S IR IT'S VERY HEARTWARMING TO SEE BUT THERE'S A TREMENDOUS SPIRIT IN THE UNITED STATES I WOULD SAY IT'S A SPIRIT LIKE
wavs/104.wav|I HAVE NEVER WITNESSED BEFORE I'VE BEEN HERE FOR A WHILE I HAVE NEVER WITNESSED THE SPIRIT THAT OUR COUNTRY HAS RIGHT NOW SO I JUST WANT TO THANK YOU ALL AND ALL OF THOSE THAT ARE POURING
wavs/105.wav|BILLIONS OF DOLLARS INTO OUR COUNTRY OR TEN DOLLARS INTO OUR COUNTRY WE THANK YOU VERY MUCH THANK YOU MISS E BESEDAND I WILL I WILL ASK YOU MAYBE ERPERSONAL QUESTION THEREBOUT
wavs/106.wav|AND PROSPEROUS AMERICA I AM HERE TO DELIVER A SIMPLE MESSAGE THERE HAS NEVER BEEN A BETTER TIME TO HIRE TO BUILD TO INVEST AND TO GROW IN THE UNITED STATES
wavs/107.wav|BEFORE DOING SO IT STOUNDS VERY UNTEERESTING A CONOLETY I DIDN'T KNOW ABOUT THIS ONE I WOULD LIKE TO ACKNOWLEDGE THE STRONG MAAM THE PESENCE OF E YOUR CAPINGTEM MBOSIES WHO
wavs/108.wav|E TREMENDOUSLY CONTRIBUTED TO TE DISCUSSIONS OR GOOD IT'S THE LAST I WOULD LIKE TO DO THAT THAT'S VERY WELL NOW STEPHEN WILBUR GARRY
wavs/109.wav|ROBERT EVEN MY GENERAL AND MY VARIOUS OTHER GENERALS YOU KNOW WE'RE MAKING OUR MILITARY PROTECTION A LITTLE BIT BETTER FOR US TOO SO THANK YOU VERY MUCH
wavs/110.wav|S EVERYBODY UNDERSTAND THAT I THINK SO THANK YOU ALL FOR BEING I'LL MY MY L MAYBE HERSON A QUESTION WOULD BE AM WHAT EXPERIENCE FROM YOUR
wavs/111.wav|PAST HAVE BEEN MOST USEFUL IN PRIPALING YOU FOR SEPRESIDENCY WELL BEING A BUSINESS MAN HAS BEEN A GREAT EXPERIENCE FOR ME I
wavs/112.wav|'VE LOVED IT I'VE ALWAYS LOVED BUSINESS I'VE ALWAYS BEEN GOOD AT BUILDING THINGS AND I'VE ALWAYS BEEN VERY SUCCESSFUL AT MAKING MONEY I'D BUISED THINGS THAT WOULD FAIL THAT WOULD BE FAILURES AND I TURN EM AROUND AND TRY AND
wavs/113.wav|ET EM FOR THE RIGHT PRICE AND THEN I TURN IM ROUND MAKE EM SUCCESSFUL AND I'VE BEEN GOOD AT IT AND THAT TAKES A CERTAIN ABILITY AND YOU KNOW HISTORICALLY I GUESS THERE'S NEVER REALLY BEEN A BUSINESS MAN OR BUSINESS PERSON
wavs/114.wav|A ELECTED PRESENT'S ALWAYS BEEN A GENERAL OR A POLITICIAN THROUGHOUT HISTORY IT'S ALWAYS BEEN A GENERAL YOU HAE TO BE A GENERAL BUT MOSTLY IT WAS POLITICIANS YOU'D NEVER HAVE A BUSINESS MAN AND
wavs/115.wav|IN ALL FAIRNESS I WAS SAYING TO TLAUS LAST NIGHT HAD THE OPPOSING PARTY TO ME ONE SOME OF WHOM YOU BACKED SOME OF THE PEOPLE IN THE ROOM INSTEAD OF BEING
wavs/116.wav|UP ALMOST FIFTY PER CENT THE STOCK MARKET IS UP SINCE MY ELECTION ALMOST FIFTY PER CENT RATHER THAN THAT I BELIEVE THE STOCK MARKET FROM THAT LEVEL THE INITIAL LEVEL WOULD HAVE BEEN DOWNFLO
wavs/117.wav|S AMERICA IS OPEN FOR BUSINESS AND WE ARE COMPETITIVE ONCE AGAIN THE AMERICAN ECONOMY IS BY FAR THE LARGEST IN THE WORLD A
wavs/118.wav|S TO FIFTY PER CENT THAT'S WHERE WE WERE HEADING I REALLY BELIEVE THAT BECAUSE THEY WEREGING TO PUT ON MASSIVE NEW REGULATIONS YOU COULDN'T BREATHE IT WAS CHOKING OUR COUNTRY TO DEATH AND I WAS ABLE TO SEE THAT GLAUSE AS A BUSINESS PURSE
wavs/119.wav|AND THE OTHER THING IS I'VE ALWAYS SEEMED TO GET FOR WHATEVER REASON A DISPROPORTIONATE AMOUNT OF PRESS OR MEDIA AND THROUGHOUT MY WHOLE LIFE SOMEBODY WILL EXPLAIN
wavs/120.wav|SOME DAY WHY BUT I'VE ALWAYS GOTTEN A LA AND AS A BUSINESS MAN I WAS ALWAYS TREATED REALLY WELL BY THE PRESS YOU KNOW THE NUMBERS SPEAK AND THINGS HAPPEN BUT I'VE ALWAYS REALLY HAD A VERY GOOD PRESS AND IT WASN'T TILL
wavs/121.wav|I BECAME A POLITICIAN THAT I REALIZED HOW NASTY HOW MEAN HOWA VICIOUS AND HOW FAKE THE PRESS CAN BE AS THE CAMERA START GOING OFF IN THE BACK
wavs/122.wav|BUT BUT OVER ALL I MEAN THE BOTTOM LINE SOMEBODY SAID WELL THEY COULDN'T HAVE BEEN THAT BED BECAUSE HERE WE ARE WE'RE PRECEDENT AND I THINK WE'RE DOING A REALLY GREAT JOB WITH MY TEAM I HAVE A TEAM OF JUS
wavs/123.wav|TREMENDOUS PEOPLE AND I THINK WE'RE DOING A VERY SPECIAL JOB AND I REALLY BELIEVE IT WAS TIME AND WAS TIME TO DO THAT JOB BECAUSE I DON'T THINK THE UNITED STATES WOUL
wavs/124.wav|HAV DONE VERY WELL IF IT WENT THROUGH FOUR OR EIGHT MORE YEARS OF REGULATION AND REALLY A VERY ANTI BUSINESS GROUP OF PEOPLE WE HAVE A VERY PRO BUSINESS GROUP WE
wavs/125.wav|HAVE A REGULATIONS CUT TO A LEVEL IN THE HISTORY OF OUR COUNTRY CLASS THIS WAS REPORTED RECENTLY IN ONE YEAR WE'VE CUT MORE REGULATIONS IN MY ADMINISTRATION THAN ANY
wavs/126.wav|ETHER ADMINISTRATION IN FOUR EIGHT OR SIXTEEN YEARS IN THE ONE CASE WE'VE GUT MORE REGULATIONS IN ONE YEAR AND WE HAVE AWAYS TO GO I WE'RE PROBABLY FIFTY PER CENT DONE AND WE'RE GOING TO HAVE
wavs/127.wav|E REGULATION THERE'S NOTHING WRONG WITH RULES AND REGULATIONS YOU NEED EM BUT WE'VE CUT MORE THAN ANY ADMINISTRATION EVER IN THE HISTORY OF OUR COUNTRY AND WE
wavs/128.wav|D WE'VE JUST ENACTED THE MOST SIGNIFICANT TAX CUTS AND REFORM IN AMERICAN HISTORY WE'VE MASSIVELY CUT TAXES FOR THE MIDDLE CLASS AND SMALL BUSINESS IS TO LET
wavs/129.wav|STILL HAVE AWAGE TO GO SO I THINK BETWEEN THAT AND THE TREMENDOUS TAX CUTS WE'VE REALLY DONE SOMETHING AND ONE OTHER THING I SAID OI I SAW IT LAST NIGHT WITH SOME OF THE LEADERS AND THE BUSINESS PEOPLE
wavs/130.wav|I THINK I'VE BEEN A CHEER LEADER FOR A COUNTRY AND EVERYBODY REPRESENTING A COMPANY OR A COUNTRY HAS TO BE A CHEER LEADER OR NO MATTER WHAT YOU DO IT'S JUST NOT GOING TO WORK
wavs/131.wav|AND THE REASON I MACHEER LEADER IS BECAUSE IT'S EASY BECAUSE I LOVE OUR COUNTRY AND I THINK WE'RE JUST DOING REALLY WELL AND WE LOOK FORWARD TO SEEING YOU IN AMERICA
wavs/132.wav|SPECIAL PLACE AND WHERE YOU ARE IS A SPECIAL PLACE ALSO THANK YOU ALL VERY MUCH I APPRECIATED THANK YOU THANK YOU VERY MUCH EH
wavs/133.wav|THE PRESIDENT FOR BEING WITH US SEWOTH ECONOMIC FORM COMMUNITY WHO IS ASSEMBLED HERE WILL BE CERTAINLY AND I QUOTE YOU FROM THE LAST PIECE OF
wavs/134.wav|YOUR REMARKS WILL BE CERDENLY BE AMONGST A HARD WORKING MEN AND WOMEN WHO DO SO DUTY EACH AND EVERY DAY MAKINGS THIS WORLD
wavs/135.wav|BETTER PLACE FOR EVERYONE THANK YOU VERY MUCH THANK YOU FOR THANK YOU VERY MUCH EVERYBODY THANK YOU
wavs/137.wav|WORKING FAMILIES KEEP MORE OF THEIR HARD EARNED MONEY WE LOWERED OUR CORPORAL TAXERATE FROM THIRTY FIVE PER CENT ALL THE WAY DOWN TO TWENTY ONE PER CENT
wavs/138.wav|AS A RESULT MILLIONS OF WORKERS HAVE RECEIVED TAX CUT BONASSES FROM THEIR EMPLOYERS IT AMOUNTS AS LARGE AS THREE THOUSAND DOLLARS THE TAX CU
wavs/139.wav|BILL IS EXPECTED TO RAISE THE AVERAGE AMERICANS HOUSEHOLD INCOME BY MORE THAN FOUR THOUSAND DOLLARS THE WORLD'S LARGEST COMPANY APPLE ANNOUNCED AT PLANS TO BRING TWO HUNDRE
wavs/140.wav|AND FORTY FIVE BILLION DOLLARS IN OVERSEAS PROFITS HOME TO AMERICA THEIR TOTAL INVESTMENT INTO THE UNITED STATES ECONOMY WILL BE MORE THAN THREE HUNDRED AND FIFT
wavs/141.wav|Y BILLION DOLLARS OVER THE NEXT FIVE YEARS NOW IS THE PERFECT TIME TO BRING YOUR BUSINESS YOUR JOBS AND YOUR INVESTMENTS TO THE UNITED STATES THI
wavs/142.wav|IS ESPECIALLY TRUE BECAUSE WE HAVE UNDERTAKEN THE MOST EXTENSIVE REGULATORY REDUCTION EVER CONCEIVED REGULATION IS
wavs/143.wav|IN BUILDING A BETTER WORLD LIKE ALL NATIONS REPRESENTED AT THIS GREAT FORUM AMERICA HOPES FOR A FUTURE IN WHICH EVERY ONE CAN PROSPER
wavs/144.wav|DEALFH TAXATION THE U S LIKE MANY OTHER COUNTRIES UNELECTED BUROCRATS AND WE HAVE BELIEVE ME WE HAVE THEM ALL OVER THE PLACE
wavs/145.wav|AND THEY'VE IMPOSED CRUSHING AND ANTI BUSINESS AND ANTI WORKER REGULATIONS ON OUR CITIZENS WITH NO VOTE NO LEGISLATIVE DEBATE AND NO REAL ACCOUNT OF
wavs/146.wav|LITY IN AMERICA THOSE DAYS ARE OVER I PLEDGED TO ELIMINATE TWO UNNECESSARY REGULATIONS FOR EVERY ONE NEW REGULATION
wavs/147.wav|WE HAVE SUCCEEDED BEYOND OUR HIGHEST EXPECTATIONS INSTEAD OF TWO FOR ONE WE HAVE CUT TWENTY TWO BURDENSOME REGULATIONS
wavs/148.wav|FOR EVERY ONE NEW RULE WE ARE FREEING OUR BUSINESSES AND WORKERS SO THEY CAN THRIVE AND FLOURISH AS NEVER BEFORE WE ARE CREATING AN ENVIRONME
wavs/149.wav|THAT ATTRACTS CAPITAL INVITES INVESTMENT AND REWARDS PRODUCTION AMERICA IS THE PLACE TO DO BUSINESS SO COME TO AMERICA WHERE YOU CAN
wavs/150.wav|INNOVATE CREATE AND BUILD I BELIEVE IN AMERICA AS PRESIDENT OF THE UNITED STATES I WILL ALWAYS PUT AMERICA FIRST
wavs/151.wav|JUST LIKE THE LEADERS OF OTHER COUNTRIES SHOULD PUT THEIR COUNTRY FIRST ALSO WBUT AMERICA FIRST DOES NOT MEAN
wavs/152.wav|AMERICA ALONE WHEN THE UNITED STATES GROWS SO DOES THE WORLD AMERICAN PROSPERITY HAS CREATED COUNTLESS JOBS ALL AROUND THE GLOW
wavs/153.wav|AND THE DRIVE FOR EXCELLENCE CREATIVITY AN INNOVATION IN THE U S HAS LED TO IMPORTANT DISCOVERIES THAT HELP PEOPLE EVERYWHERE LIVE MORE PROSPEROUS AND FAR
wavs/154.wav|AND EVERY CHILD CAN GROW UP FREE FROM VIOLENCE POVERTY AND FEAR OVER THE PAST YEAR WE HAVE MADE EXTRAORDINARY STRIDES IN THE U S WHERE
wavs/155.wav|R HEALTHIER LIVES AS THE UNITED STATES PURSUES DOMESTIC REFORMS TO UNLEASE JOBS AND GROWTH WE ARE ALSO WORKING TO REFORM THE INTERNATIONAL
wavs/156.wav|RATING SYSTEM SO THAT IT PROMOTES BROADLY SHARED PROSPERITY AND REWARDS TO THOSE WHO PLAY BY THE RULES WE CANNOT HAVE FREE AND A
wavs/157.wav|IN TRADE IF SOME COUNTRIES EXPLOIT THE SYSTEM AT THE EXPENSE OF OTHERS WE SUPPORT FREE TRADE BUT IT NEEDS TO BE FAIR
wavs/158.wav|AND IT NEEDS TO BE RECIPROCAL BECAUSE IN THE END UNFAIR TRADE UNDERMINES US ALL THE UNITED STATES WILL NO LONGER TURN A BLIND EYE TO ON
wavs/159.wav|FAR ECONOMIC PRACTICES INCLUDING MASSIVE INTELLECTUAL PROPERTY THEFT INDUSTRIAL SUBSIDIES AND PERVASIVE STATE LED ECONOMIC PLANNING
wavs/160.wav|THESE AND OTHER PREDATORY BEHAVIORS ARE DISTORTING THE LOBAL MARKETS AND HARMING BUSINESSES AND WORKERS NOT JUST IN THE U S BUT AROUND THE GLUV
wavs/161.wav|JUST LIKE WE EXPECT THE LEADERS OF OTHER COUNTRIES TO PROTECT THEIR INTERESTS AS PRESIDENT OF THE UNITED STATES I WILL ALWAYS PROTECT THE INTERESTS OF OUR COUNTRY OUR
wavs/162.wav|COMPANIES AND OUR WORKERS WE WILL ENFORCE OUR TRADE LAWS AND RESTORE INTEGRITY TO OUR TRADING SYSTEM ONLY BY INSISTING ON FAIR AND RECIPROCAL
wavs/163.wav|TRADE CAN WE CREATE A SYSTEM THAT WORKS NOT JUST FOR THE U S BUT FOR ALL NATIONS AS I HAVE SAID THE UNITED STATES IS PREPARED TO NEGOTIATE
wavs/164.wav|MUTUALLY BENEFICIAL BY LATERAL TRADE AGREEMENTS WITH ALL COUNTRIES THIS WILL INCLUDE THE COUNTRIES IN T P P WHICH ARE VERY IMPORTANT W
wavs/165.wav|LIFTING UP FORGOTTEN COMMUNITIES CREATING EXCITING NEW OPPORTUNITIES AND HELPING EVERY AMERICAN FIND THEIR PATH TO THE AMERICAN DREAM THE DREAM OF A GREAT JOB A SAFE HO
wavs/166.wav|E HAVE AGREEMENTS WITH SEVERAL OF THEM ALREADY WE WOULD CONSIDER NEGOTIATING WITH THE REST EITHER INDIVIDUALLY OR PERHAPS AS A GROUP IF IT IS IN THE INTERESTS OF
wavs/167.wav|ALL MY ADMINISTRATION IS ALSO TAKING SWIFT ACTION IN OTHER WAYS TO RESTORE AMERICAN CONFIDENCE AND INDEPENDENCE WE ARE LIF
wavs/168.wav|TING SELF IMPOSED RESTRICTIONS ON ENERGY PRODUCTION TO PROVIDE A FORDABLE POWER TO OUR CITIZENS AND BUSINESSES AND TO PROMOTE ENERGY SECURITY FOR OUR FRIEN
wavs/169.wav|S ALL AROUND THE WORLD NO COUNTRY SHOULD BE HELD HOSTAGE TO A SINGLE PROVIDER OF ENERGY AMERICA IS ROARING BACK AND NOW IS THE T
wavs/170.wav|ME TO INVEST IN THE FUTURE OF AMERICA WE HAVE DRAMATICALLY CUT TAXES TO MAKE AMERICA COMPETITOR WE ARE ELIMINATING BURDENSOME REGULATIONS AT A RECORD PAY
wavs/171.wav|S WE ARE REFORMING THE BUREOCRASY TO MAKE IT LEAN RESPONSIVE AND ACCOUNTABLE AND WE ARE ENSURING OUR LAWS ARE ENFORCED FAIRLY
wavs/172.wav|WE HAVE THE BEST COLLEGES AND UNIVERSITIES IN THE WORLD AND WE HAVE THE BEST WORKERS IN THE WORLD ENERGY IS ABUNDANT AND AFFORDABLE THERE HAS NEVER BEEN A BETTER TIME TO DO BUSINESS
wavs/173.wav|IN AMERICA WE ARE ALSO MAKING HISTORIC INVESTMENTS IN THE AMERICAN MILITARY BECAUSE WE COULD NOT HAVE PROSPERITY WITHOUT SECURITY TO MAKE THE WORLD SAFER
wavs/174.wav|FROM ROGUE REGIMES TERRORISM AND REVISIONISTS POWERS WE ARE ASKING OUR FRIENDS AND ALLIES TO INVEST IN THEIR OWN DEFENCES AND TO MEET THEIR FINANCIAL OBLIG
wavs/175.wav|ATIONS OUR COMMON SECURITY REQUIRES EVERY ONE TO CONTRIBUTE THEIR FAIR SHARE MY ADMINISTRATION IS PROUD TO HAVE LED HISTORIC EFFORTS AT THE UNITED NATIONS
wavs/176.wav|M AND A BETTER LIFE FOR THEIR CHILDREN AFTER YEARS OF STAGNATION THE UNITED STATES IS ONCE AGAIN EXPERIENCING STRONG ECONOMIC GROWTH
wavs/177.wav|ECURITY COUNCIL AND ALL AROUND THE WORLD TO UNITE ALL CIVILIZED NATIONS IN OUR CAMPAIGN OF MAXIMUM PRESSURE TO DENUK THE KOREAN
wavs/178.wav|PENINSULA WE CONTINUE TO CALL ON PARTNERS TO CONFRONT A RAN SUPPORT FOR TERRACE AND BLOCK A RAN'S PATH TO A NEWCLEAR WEAPON WE'RE ALSO WORKING WITHO
wavs/179.wav|LIS AND PARTNERS TO DESTROY GEHATTES TERRACED ORGANIZATION SUCH AS ISIS AND VERY SUCCESSFULLY SO THE UNITED STATES IS LEADING A VERY BROAD COALITION TO
wavs/180.wav|ENY TERRORS CONTROL OF THEIR TERRITORY AND POPULATIONS TO CUT OFF THEIR FUNDING AND TO DISCREDIT THEIR WICKED IDEOLOGY I AM PLEASED TO REPORT THAT THE COALI
wavs/181.wav|TION TO DEFEAT ISIS HAS RETAKEN ALMOST ONE HUNDRED PER CENT OF THE TERRITORY ONCE HELD BY THESE KILLERS IN ERAC AND SYRIA THERE IS STIL
wavs/182.wav|LL MORE FIGHTING AND WORK TO BE DONE AND TO CONSOLIDATE OUR GAINS WE ARE COMMITTED TO ENSURING THAT AFGANISTAN NEVER AGAIN
wavs/183.wav|BECOMES A SAFE HAVEN FARTERAS WHO WANT TO COMMIT MASS MURDER TO OUR CIVILIAN POPULATIONS I WANT TO THANK THOSE NATIONS REPRESENTED HERE TO DAY THAT HAVE
wavs/184.wav|JOINED IN THESE CRUCIAL EFFORTS YOU ARE NOT JUST SECURING YOUR OWN CITIZENS BUT SAVING LIVES AND RESTORING HOPE FOR MILLIONS AND MILLIONS OF PEOPLE
wavs/185.wav|WHEN IT COMES TO TERRORISM WE WILL DO WHATEVER IS NECESSARY TO PROTECT OUR NATION WE WILL DEFEND OUR CITIZENS AND OUR BORDERS WE ARE ALSO SECURING OUR IMMI
wavs/186.wav|GRATION SYSTEM IS A MATTER OF BOTH NATIONAL AND ECONOMIC SECURITY AMERICA IS A CUTTING EDGE ECONOMY WAT OUR EMIGRATION SYSTEM
wavs/187.wav|THE STOCK MARKET IS SMASHING ONE RECORD AFTER ANOTHER AND HAS ADDED MORE THAN SEVEN TRILLION DOLLARS IN NEW WEALTH SINCE MY ELECTION
wavs/188.wav|IS STUCK IN THE PAST WE MUST REPLACE OUR CURRENT SYSTEM OF EXTENDED FAMILY CHAIN MIGRATION WITH A MERIT BASE SYSTEM OF ADMISSIONS THAT SELECTS
wavs/189.wav|NEW ARRIVALS BASED ON THEIR ABILITY TO CONTRIBUTE TO OUR ECONOMY TO SUPPORT THEMSELVES FINANCIALLY AND TO STRENGTHEN OUR COUNTRY IN REBUILDING AMERICA WE A
wavs/190.wav|RE ALSO FULLY COMMITTED TO DEVELOPING OUR WORK FORCE WE ARE LIFTING PEOPLE FROM DEPENDENCE TO INDEPENDENCE BECAUSE WE KNOW THE SINGLE BEST ANTI POVERTY PROGRAMM
wavs/191.wav|M IS A VERY SIMPLE AND VERY BEAUTIFUL PECHENK TO BE SUCCESSFUL IT IS NOT ENOUGH TO INVEST IN OUR ECONOMY WE MUST
wavs/192.wav|NVEST IN OUR PEOPLE WHEN PEOPLE ARE FORGOTTEN THE WORLD BECOMES FRACTURED ONLY BY HEARING AND RESPONDING TO THE VOICES OF THE FOR
wavs/193.wav|GOTTEN CAN WE CREATE A BRIGHT FUTURE THAT IS TRULY SHARED BY ALL THE NATION'S GREATNESS IS MORE THAN THE SUM OF ITS PRODUCTION A NATION'S GREATNESS
wavs/194.wav|IS THE SUM OF ITS CITIZENS THE VALUES PRIDE LOVE DEVOTION AND CHARACTER OF THE PEOPLE WHO CALL THAT NATION HOME FROM
wavs/195.wav|MY FIRST INTERNATIONAL G SEVENTH SUMMIT TO THE G TWENTY TO THE U N GENERAL ASSEMBLY TO APEC TO THE WORLD TRADE ORGANIZATION AND TODAY AT THE WORLD ECONOMIC
wavs/196.wav|FORUM MY ADMINISTRATION HAS NOT ONLY BEEN PRESENT BUT HAS DRIVEN OUR MESSAGE THAT WE ARE ALL STRONGER WHEN FREE SOVEREIGN NATIONS COOPERATE
wavs/197.wav|RD SHARED GIRLS AND THEY COOPERATE TOWARD SHARED DREAMS REPRESENTED IN THIS ROOM ARE SOME OF THE REMARKABLE CITIZENS FROM ALL OVER THE WORLD YOU ARE NATH
wavs/198.wav|ONSUMER CONFIDENCE BUSINESS CONFIDENCE AND MANUFACTURING CONFIDENCE ARE THE HIGHEST THEY HAVE BEEN IN MANY DECADES SINCE BY ELECTION
wavs/199.wav|NAL LEADERS BUSINESS TITANS INDUSTRY GIANTS AND MANY OF THE BRIGHTEST MINES IN MANY FIELDS EACH OF YOU HAS THE POWER TO CHANGE HEARTS
wavs/200.wav|TRANSFORM LIVES AND SHAPE YOUR COUNTRY'S DESTINIES WITH THIS POWER COMES AN OBLIGATION HOWEVER A DUTY OF LOYALTY TO THE PEOPLE WORKERS AND CUSTOMERS WHO
wavs/201.wav|VE MADE YOU WHO YOU ARE SO TOGETHER LET US RESOLVE TO USE OUR POWER OUR RESOURCES AND OUR VOICES NOT JUST FOR OURSELVES BUT
wavs/202.wav|OR OUR PEOPLE TO LIFT THEIR BURDENS TO RAISE THEIR HOPES AND TO EMPOWER THEIR DREAMS TO PROTECT THEIR FAMILIES THEIR COMMUNITIES THEIR HISTORIES AND THEIR FUTURES
wavs/203.wav|THAT'S WHAT WE'RE DOING IN AMERICA AND THE RESULTS ARE TOTALLY UNMISTAKABLE IT'S WHY NEW BUSINESSES AND INVESTMENT ARE FLOODING IN IT'S W
wavs/204.wav|Y OUR UNEMPLOYMENT RATE IS THE LOWEST IT'S BEEN IN SO MANY DECADES IT'S WHY AMERICA'S FUTURE HAS NEVER BEEN BRIGHTER TO DAY I AM INVITING ALL OF YOU T
wavs/205.wav|BECOME PART OF THIS INCREDIBLE FUTURE WE ARE BUILDING TOGETHER THANK YOU TO OUR HOSTS THANK YOU TO THE LEADERS AND INNOVATORS IN THE AUDIENCE BUT MOST IMPORTANTLY THANK YOU TO AL
wavs/206.wav|LE OF THE HARD WORKING MEN AND WOMEN WHO DO THEIR DUTY EACH AND EVERY DAY MAKING THIS A BETTER WORLD FOR EVERYONE TOGETHER LET US SEND OUR LOVE
wavs/207.wav|AND OUR GRATITUDE TO MAKE THEM BECAUSE THEY REALLY MAKE OUR COUNTRIES RUN THEY MAKE OUR COUNTRIES GREAT THANK YOU
wavs/208.wav|AND GOD BLESS YOU ALL THANK YOU VERY MUCH
wavs/209.wav|E'VE CREATED TWO POINT FOUR MILLION JOBS AND THAT NUMBER IS GOING UP VERY VERY SUBSTANTIALLY SMALL BUSINESS OPTIMISM IS AT AN OLD TIME HIGH"""

lines = data.strip().split('\n')
files = []
# Extract the filename and text part
for line in lines:
    filename, sentence = line.split('|')
    output_filename = filename.replace("wavs/", "")
    files+=[output_filename]
print(files)


# Resample function to ensure both audios have the same sample rate (16kHz)
def resample_audio(audio, original_sample_rate, target_sample_rate=16000):
    if original_sample_rate != target_sample_rate:
        transform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
        audio = transform(audio)
    return audio

Pesq_scores = []
Stoi_scores = []
llr_scores = []
wss_scores = []
snrseg_scores = []
csig_scores = []
cbak_scores = []
covl_scores = []

for i in files:
    # Load predicted and original audio files
    predicted, sample_rate_predicted = torchaudio.load(f"output_wavs/{i}")
    original, sample_rate_original = torchaudio.load(f"trump/wavs/{i}")

    # Resample to 16kHz if needed
    predicted = resample_audio(predicted, sample_rate_predicted)
    original = resample_audio(original, sample_rate_original)

    # Convert stereo to mono if necessary
    if predicted.shape[0] > 1:
        predicted = torch.mean(predicted, dim=0).unsqueeze(0)
    if original.shape[0] > 1:
        original = torch.mean(original, dim=0).unsqueeze(0)

    min_length = min(predicted.shape[1], original.shape[1])
    predicted = predicted[:, :min_length]
    original = original[:, :min_length]

    # Convert tensors to NumPy arrays before calling pysepm functions
    predicted_np = predicted.cpu().numpy().squeeze()
    original_np = original.cpu().numpy().squeeze()

    # PESQ and STOI scores
    pesq_score = pesq(predicted, original)
    Pesq_scores.append(pesq_score)

    stoi_score = stoi(predicted, original)
    Stoi_scores.append(stoi_score)

    # Compute LLR
    llr_score = pysepm.llr(predicted_np, original_np, fs=16000)
    llr_scores.append(llr_score)

    # Compute WSS
    wss_score = pysepm.wss(predicted_np, original_np, fs=16000)
    wss_scores.append(wss_score)

    # Compute Segmental SNR
    snrseg_score = pysepm.SNRseg(predicted_np, original_np, fs=16000)
    snrseg_scores.append(snrseg_score)

    # Compute CSIG
    csig = 3.093 - 1.029 * llr_score + 0.603 * pesq_score - 0.009 * wss_score
    csig_scores.append(csig)

    # Compute CBAK
    cbak = 1.634 + 0.478 * pesq_score - 0.007 * wss_score + 0.063 * snrseg_score
    cbak_scores.append(cbak)

    # Compute COVL
    covl = 1.594 + 0.805 * pesq_score - 0.512 * llr_score - 0.007 * wss_score
    covl_scores.append(covl)

print(f"Average PESQ score: {sum(Pesq_scores) / len(Pesq_scores)}")
print(f"Average STOI score: {sum(Stoi_scores) / len(Stoi_scores)}")
print(f"Average csig score: {sum(csig_scores) / len(csig_scores)}")
print(f"Average cbak score: {sum(cbak_scores) / len(cbak_scores)}")
print(f"Average covl score: {sum(covl_scores) / len(covl_scores)}")

['1.wav', '2.wav', '3.wav', '4.wav', '5.wav', '6.wav', '7.wav', '8.wav', '9.wav', '10.wav', '11.wav', '12.wav', '13.wav', '14.wav', '15.wav', '16.wav', '17.wav', '18.wav', '19.wav', '20.wav', '21.wav', '22.wav', '23.wav', '24.wav', '25.wav', '26.wav', '27.wav', '28.wav', '29.wav', '30.wav', '31.wav', '32.wav', '33.wav', '34.wav', '35.wav', '36.wav', '37.wav', '38.wav', '39.wav', '40.wav', '41.wav', '42.wav', '43.wav', '44.wav', '45.wav', '46.wav', '47.wav', '48.wav', '49.wav', '50.wav', '51.wav', '52.wav', '53.wav', '54.wav', '55.wav', '56.wav', '57.wav', '58.wav', '59.wav', '60.wav', '61.wav', '62.wav', '63.wav', '64.wav', '65.wav', '66.wav', '67.wav', '68.wav', '69.wav', '70.wav', '71.wav', '72.wav', '73.wav', '74.wav', '75.wav', '76.wav', '77.wav', '78.wav', '79.wav', '80.wav', '81.wav', '82.wav', '83.wav', '84.wav', '85.wav', '86.wav', '87.wav', '88.wav', '89.wav', '90.wav', '91.wav', '92.wav', '93.wav', '94.wav', '95.wav', '96.wav', '97.wav', '98.wav', '99.wav', '100.wav', '101.wa